In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv
import datetime
import re

In [2]:
# # set here to provide data to test code

# my_date = datetime.date.today() 
# year, week_num, day_of_week = my_date.isocalendar()
# this_week = week_num

# url_for_scrape = 'https://www.metacritic.com/browse/albums/release-date/new-releases/date'
# user_agent = {'User-agent': 'Mozilla/5.0'}
# # send response
# response_score = requests.get(url_for_scrape, headers = user_agent)
# # scrape website into variable to parse
# soup_score = BeautifulSoup(response_score.text, 'html.parser')

# # create temporary lists for user scores
# userP = []
# userM = []
# userN = []
# # create/initialize dictionary 
# albums_dict = {'artist':[], 'album':[], 'date':[], 'week_num':[], 'meta_score': [], 'user_score':[]}

# soup_score.find_all('td', class_='clamp-summary-wrap')
# # create soup 
# for _ in soup_score.find_all('td', class_='clamp-summary-wrap'):
#     # scrape album name
#     albums_dict['album'].append(_.find('a', class_= 'title').text)
#     # scrape artist name and strip white space and extra characters
#     albums_dict['artist'].append(_.find('div', class_='artist').text.strip().lstrip('by '))
#     # scrape date
#     albums_dict['date'].append(_.find('div', class_='clamp-details').find('span').text)
#     # scrape meta_score, handle for changes in class name, convert data type of score to int and append to dict
#     # except set to pass since all alubms have a score

#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release positive').text))  
#     except:
#         pass
#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release mixed').text))  
#     except:
#         pass 
#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release negative').text))  
#     except:
#         pass
#     # scrape user score, handle errors for tbd/class name and append to temp list
#     try:
#         userP.append(float(_.find('div', class_='metascore_w user large release positive').text))  
#     except:
#         userP.append(0)
#     try:
#         userM.append(float(_.find('div', class_='metascore_w user large release mixed').text))  
#     except:
#         userM.append(0)
#     try:
#         userN.append(float(_.find('div', class_='metascore_w user large release negative').text))  
#     except:
#         userN.append(0)

# # merge user score by filtering scores from tbd using data type in temporary lists, convert data type of scores to int and append to dictionary        
# for a, b, c in zip(userP, userM, userN):
#     if isinstance(a, float):
#         albums_dict['user_score'].append(int(a * 10))
#     elif isinstance(b, float):
#         albums_dict['user_score'].append(int(b * 10))
#     elif isinstance(c, float):
#         albums_dict['user_score'].append(int(c * 10))
#     else:
#         albums_dict['user_score'].append(c)
# # create week_num key and values for weekly scrape
# for dates in albums_dict['date']:
#     albums_dict['week_num'].append((datetime.datetime.strptime(dates, '%B %d, %Y')).isocalendar()[1])
# # write dictionary to csv
# # create header
# fields = ['artist', 'album', 'date', 'week_num', 'meta_score', 'user_score'] 
# # create variable for data to be written
# data = zip(albums_dict['artist'], albums_dict['album'], albums_dict['date'], albums_dict['week_num'], albums_dict['meta_score'], albums_dict['user_score'])
# output_path = os.path.join('..', 'data', 'meta_scrape.csv')
# # create header
# fields = ['artist', 'album', 'date', 'week_num', 'meta_score', 'user_score'] 
# # create variable for data to be written
# data = zip(
#         albums_dict['artist'], 
#         albums_dict['album'], 
#         albums_dict['date'], 
#         albums_dict['week_num'], 
#         albums_dict['meta_score'], 
#         albums_dict['user_score']
#         )
# with open(output_path, 'w') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(fields)
#     for d in data:
#         writer.writerow(d)



In [3]:
file_path = os.path.join('..', 'data', 'meta_scrape.csv')

In [4]:
scrape_df = pd.read_csv(file_path)  
# dedup df        
# scrape_clean = scrape_df.drop_duplicates(subset = 'artist', ignore_index = True)
# scrape_clean
scrape_df

,artist,album,date,week_num,meta_score,user_score
0,Snail Mail,Valentine,"November 5, 2021",44,88,0
1,The War on Drugs,I Don't Live Here Anymore,"October 29, 2021",43,85,79
2,Mastodon,Hushed and Grim,"October 29, 2021",43,83,82
3,Geese,Projector,"October 29, 2021",43,87,93
4,Ed Sheeran,Equals,"October 29, 2021",43,57,58
...,...,...,...,...,...,...
95,Mickey Guyton,Remember Her Name,"September 24, 2021",38,79,0
96,Alessia Cara,In the Meantime,"September 24, 2021",38,80,94
97,RP Boo,Established!,"September 17, 2021",37,79,0
98,Billy Idol,The Roadside [EP],"September 17, 2021",37,68,0


In [5]:


def create_review_url(df):
###
# takes a df latest metascrape, cleans the artist and album strings
# and adds a new column of urls to scrape review values
###
    review_urls = []
    # iterate over dataframe
    for i, j in df.iterrows():
    #     clean album and artist names of all nonalpha except for ! and accent marks
        al = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', j['album'])
        ar = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', j['artist'])
    # correct instances of multilple spaces
        al = re.sub(' +', ' ', al)
        ar = re.sub(' +', ' ', ar)
    # make lowercase and join words with - for url
        url_end = f'{al}/{ar}'.replace(" ", "-").lower() 
    # concat url for review scrape    
        url_beginning ='https://www.metacritic.com/music/'
        review_urls.append(url_beginning + url_end)
# get value of colmuns to use for new column position
    new_col_position = len(df.columns)
# Use insert to add review_urls column and values to dataframe
    df.insert(new_col_position, "review_urls", review_urls)
    return df

In [6]:
scrape_url_df = create_review_url(scrape_df)


In [102]:
scrape_url_df
###
# found an errant / in user reviews and column is of datatype string and don't know why
# this fixes that but need to check scrape function to see why
###
# for _ in scrape_clean_url_df['user_score']:
#     if len(_) > 2:
#         _=_[:2]
#     print(_)
# print(scrape_clean_url_df['user_score'])       
# scrape_clean_url_df = int(scrape_clean_url_df['user_score'])

,artist,album,date,week_num,meta_score,user_score,review_urls
0,Snail Mail,Valentine,"November 5, 2021",44,88,0,https://www.metacritic.com/music/valentine/sna...
1,The War on Drugs,I Don't Live Here Anymore,"October 29, 2021",43,85,79,https://www.metacritic.com/music/i-dont-live-h...
2,Mastodon,Hushed and Grim,"October 29, 2021",43,83,82,https://www.metacritic.com/music/hushed-and-gr...
3,Geese,Projector,"October 29, 2021",43,87,93,https://www.metacritic.com/music/projector/geese
4,Ed Sheeran,Equals,"October 29, 2021",43,57,58,https://www.metacritic.com/music/equals/ed-she...
...,...,...,...,...,...,...,...
95,Mickey Guyton,Remember Her Name,"September 24, 2021",38,79,0,https://www.metacritic.com/music/remember-her-...
96,Alessia Cara,In the Meantime,"September 24, 2021",38,80,94,https://www.metacritic.com/music/in-the-meanti...
97,RP Boo,Established!,"September 17, 2021",37,79,0,https://www.metacritic.com/music/established!/...
98,Billy Idol,The Roadside [EP],"September 17, 2021",37,68,0,https://www.metacritic.com/music/the-roadside-...


In [111]:
def scrape_reviews(df):
    ###
    # takes a meta_scrape dataframe with review urls and 
    # scrapes the num of reviews-both critics and users-
    # album image, record label and first genre of album
    # and returns a dataframe
    ###
    img = []
    crit_num = []
    user_num = []
    label = []
    genre = []

    for i, j in df.iterrows():
        url = j['review_urls']
        user_agent = {'User-agent': 'Mozilla/5.0'}
        # send response
        response_reviews = requests.get(url, headers = user_agent)
        # scrape website into variable to parse
        soup_reviews = BeautifulSoup(response_reviews.text, 'html.parser')
        ########### comment back in from here
        # scrape album image 
        img_soup = soup_reviews.find('meta', property="og:image")
        img.append(img_soup['content'])
        # scrape num of critical reviews
        num_rev=(soup_reviews.find('span', itemprop="reviewCount"))
        crit_num.append(num_rev.text.strip())
        # scrape num of user reviews
        if j['user_score'] == 0:
            user_num.append(0)
        else:
            try:
                user_revs = float(soup_reviews.find('div', class_='metascore_w user large album positive').text)
                user_num.append(int(user_revs*10))
                countpos+=1
            except:
                pass
            try:
                user_revs = float(soup_reviews.find('div', class_='metascore_w user large album mixed').text)
                user_num.append(int(user_revs*10))
                countmix+=1
            except:
                pass
            try:
                user_revs = float(soup_reviews.find('div', class_='metascore_w user large album negative').text)
                user_num.append(int(user_revs*10))
                countneg+=1
            except:
                pass
        # scrape record labels
        label_class = soup_reviews.find_all("span", itemprop="name")
        label.append(label_class[2].text.strip())
        # scrape genre
        genre.append(soup_reviews.find("span", itemprop="genre").text)
    # drop column for url and add scrapped lists to dataframe
    df = df.drop(columns='review_urls').assign(album_img = img, crit_rev_num = crit_num, user_rev_num = user_num, record_label = label, album_genre = genre)
    return df



In [112]:
test_df = scrape_reviews(scrape_url_df)

In [116]:
test_df.groupby(by='week_num').count()

,artist,album,date,meta_score,user_score,album_img,crit_rev_num,user_rev_num,record_label,album_genre
week_num,,,,,,,,,,
37,3,3,3,3,3,3,3,3,3,3
38,14,14,14,14,14,14,14,14,14,14
39,20,20,20,20,20,20,20,20,20,20
40,12,12,12,12,12,12,12,12,12,12
41,18,18,18,18,18,18,18,18,18,18
42,20,20,20,20,20,20,20,20,20,20
43,12,12,12,12,12,12,12,12,12,12
44,1,1,1,1,1,1,1,1,1,1


In [ ]:
# Making a Connection with MongoClient
client = MongoClient("mongodb://localhost:27018/")
# database
db = client["stocks_database"]
# collection
company= db["Company"]